# **Analyze the data from scratch**
steps

## Load Libraries

In [2]:
# LOAD LIBRARIES
library(Seurat)
library(tidyverse)
library(future)
library(ggplot2)
library(dplyr)
library(presto)
library(cowplot)

In [4]:
#install.packages("openai")
#remotes::install_github("Winnie09/GPTCelltype")
#install.packages("enrichR")

# Cell annotation
library(enrichR)
library(GPTCelltype)
library(openai)

## Setup

In [5]:
# SET UP NAMES
timepoints <- c("23days", "1month", "1.5month", "2month", "3month", "4month", "5month", "6month")
housekeeping_genes <- c("ACTB", "DLG4")
genes_of_interest <- c("SRCIN1", "KIAA1217", "CIT")
path_to_data <- "/sharedFolder/data_organoidi_velasco/Data/"

name_new_dir_results <- paste(getwd(), "/Results", sep = "")
if (!dir.exists(name_new_dir_results)) {
    dir.create(name_new_dir_results)
}

name_new_dir_partial <- paste(getwd(), "/Partial", sep = "")
if (!dir.exists(name_new_dir_partial)) {
    dir.create(name_new_dir_partial)
}

In [6]:
for (a in 1:8) {
    name_new_dir <- paste(name_new_dir_partial, "/", timepoints[a], sep="")     
    if (!dir.exists(name_new_dir)) {
        dir.create(name_new_dir)
    }
    name_new_dir <- paste(name_new_dir_results, "/", timepoints[a], sep="")     
    if (!dir.exists(name_new_dir)) {
        dir.create(name_new_dir)
    }  
}

## Define functions

### load.data(time_point)
* LOAD DATA
* NORMALIZE
* FIND VARIABLE FEATURES
* SCALE DATA

In [13]:
load.data <- function(time_point) {
    print(paste("Loading data for time point:", timepoints[time_point]))

    # Load the data
    sc_data <- Read10X(data.dir = paste(path_to_data, "expression_", timepoints[time_point], sep = ""), gene.column = 1)

    # Create Seurat object
    sc_data <- CreateSeuratObject(counts = sc_data, min.cells = 3, min.features = 500, project = timepoints[time_point], names.delim = "-", names.field = 2)

    # Normalize the data
    sc_data <- NormalizeData(sc_data, normalization.method = "LogNormalize", scale.factor = 1e6)

    # Find variable features
    sc_data <- FindVariableFeatures(sc_data, selection.method = "mvp", nfeatures = 2000)

    # Scale the data
    sc_data <- ScaleData(sc_data)

    # Save the Scaled data
    name_new_dir <- paste(name_new_dir_partial, "/", timepoints[time_point], sep = "")
    if (!dir.exists(name_new_dir)) {
        dir.create(name_new_dir)
    }
    print(paste("Saving PCA for time point", timepoints[time_point], "in", name_new_dir))
    save(sc_data, file = paste(name_new_dir, "/Scaled_", timepoints[time_point], ".Robj", sep = ""))

    return(sc_data)
}

### PCA.cluster(sc_data, res)
* Run PCA
* Cluster the cells

In [14]:
PCA.cluster <- function(sc_data, res){
    print(paste("Running PCA and clustering for time point:", timepoints[time_point]))
    
    # PCA
    sc_data <- RunPCA(sc_data, npcs = 50, verbose = FALSE)
    #print(ElbowPlot(object = sc_data, ndims = 50))

    # Cluster the cells
    sc_data <- FindNeighbors(sc_data, dims = 1:40)
    sc_data <- FindClusters(sc_data, resolution = res)
    
    print(table(Idents(sc_data)))

    # Save the PCA plot
    name_new_dir <- paste(name_new_dir_partial, "/", timepoints[time_point], sep="")
    if (!dir.exists(name_new_dir)) {
        dir.create(name_new_dir)
    } 

    print(paste("Saving PCA for time point", timepoints[time_point], "in", name_new_dir))
    save(sc_data, file = paste(name_new_dir, "/PCA_res_",res,"_",timepoints[time_point],".Robj", sep=""))

    return(sc_data)
}

### cluster.markers(sc_data)
* find all markers for every cluster compared to all remaining cells

In [9]:
# FIND ALL MARKERS
cluster.markers <- function(sc_data) {
    print(paste("Finding all markers for time point:", timepoints[time_point]))

    # Find all markers for every cluster compared to all remaining cells
    cluster_markers <- FindAllMarkers(sc_data,
                                        only.pos = TRUE,   # Considera solo i marker espressi positivamente
                                        min.pct = 0.25,    # Percentuale minima di espressione nelle cellule del cluster
                                        logfc.threshold = 0.25)  # Soglia minima di LogFC
    
    # Save the markers
    name_new_dir <- paste(name_new_dir_partial, "/", timepoints[time_point], sep="")     
    if (!dir.exists(name_new_dir)) {
        dir.create(name_new_dir)
    } 
    
    print(paste("Saving cluster markers for time point", timepoints[time_point], "in", name_new_dir))
    save(cluster_markers, file = paste(name_new_dir, "/cluster_markers_",timepoints[time_point],".Robj", sep=""))

    return(cluster_markers)
}

In [10]:
# FIND CONSERVED MARKERS
#FindConservedMarkers

### Reload from partials
* load.sc_data = up to scaleData()
* load.cluster = clustered
* load.markers = markers

In [11]:
# RELOAD DATA
load.sc_data <- function(time_point) {
    name_new_dir <- paste(name_new_dir_partial, "/", timepoints[time_point], sep = "")
    load(paste(name_new_dir, "/cluster_markers_", timepoints[time_point], ".Robj", sep = ""))
    return(sc_data)
}

load.clusters <- function(time_point, res) {
    name_new_dir <- paste(name_new_dir_partial, "/", timepoints[time_point], sep = "")
    load(paste(name_new_dir, "/PCA_res_", res, "_", timepoints[time_point], ".Robj", sep = ""))
    return(sc_data)
}

load.markers <- function(time_point) {
    name_new_dir <- paste(name_new_dir_partial, "/", timepoints[time_point], sep = "")
    load(paste(name_new_dir, "/cluster_markers_", timepoints[time_point], ".Robj", sep = ""))
    return(cluster_markers)
}

### de.genes(genes_oi)
* finds whether one or more genes of interest are differentialy expressed in the clusters
* if no genes are specified this function assumes that **genes_oi** is **genes_of_interest**

In [19]:
# FIND DIFFERENTIALLY EXPRESSED GENES
de.genes <- function(genes_oi = genes_of_interest) {
    print(paste("Finding differentially expressed genes for time point:", timepoints[time_point]))

    # Find differentially expressed genes
    de_genes <- cluster_markers %>% filter(gene %in% genes_of_interest)
    print(de_genes)

    # Save the DE genes
    name_new_dir <- paste(name_new_dir_results, "/", timepoints[time_point], sep = "")
    if (!dir.exists(name_new_dir)) {
        dir.create(name_new_dir)
    }
    print(paste("Saving differentially expressed genes for time point", timepoints[time_point], "in", name_new_dir))
    write.csv(de_genes, file = paste(name_new_dir, "/de_genes_", timepoints[time_point], ".csv", sep = ""))

    return(de_genes)
}

# Compute

## Operations on one dataset

### Generate clusters 

In [65]:
do.iteration <- function(time_point){
    print(paste("Processing time point:", timepoints[time_point]))

    # LOAD LIBRARIES
    library(Seurat)
    library(tidyverse)
    library(future)
    library(ggplot2)
    library(dplyr)
    library(presto)
    #library(enrichR)
    library(cowplot)

    # Load the data
    sc_data <- load_data(time_point)
    
    # Run PCA and clustering
    sc_data <- PCA_cluster(sc_data, res = 1)
    
    # Find all markers
    cluster_markers <- cluster_markers(sc_data)
}

In [43]:
do_iteration(1)

[1] "Processing time point: 23days"
[1] "Loading data for time point: 23days"


Normalizing layer: counts

Finding variable features for layer data

Centering and scaling data matrix



[1] "Running PCA and clustering for time point: 23days"


Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 29736
Number of edges: 1146654

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9253
Number of communities: 18
Elapsed time: 5 seconds

   0    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15 
7414 2507 2386 2257 2102 2024 2020 1909 1656 1554 1458 1021  475  400  246  116 
  16   17 
 106   85 
[1] "Finding all markers for time point: 23days"


Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Calculating cluster 8

Calculating cluster 9

Calculating cluster 10

Calculating cluster 11

Calculating cluster 12

Calculating cluster 13

Calculating cluster 14

Calculating cluster 15

Calculating cluster 16

Calculating cluster 17



[1] "Finding differentially expressed genes for time point: 23days"


ERROR: Error in UseMethod("filter"): no applicable method for 'filter' applied to an object of class "function"


In [49]:
time_point <- 1
print(timepoints[time_point])

[1] "23days"


In [50]:
# Load the data
sc_data <- load_data(time_point)

[1] "Loading data for time point: 23days"


Normalizing layer: counts

Finding variable features for layer data

Centering and scaling data matrix



In [52]:
# Run PCA and clustering
sc_data <- PCA_cluster(sc_data, res = 1)

[1] "Running PCA and clustering for time point: 23days"


Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 29736
Number of edges: 1146654

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8914
Number of communities: 25
Elapsed time: 6 seconds

   0    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15 
5195 2314 2018 2001 1731 1659 1558 1476 1446 1341 1213 1185 1021  914  872  585 
  16   17   18   19   20   21   22   23   24 
 578  545  526  488  401  362  116  106   85 


### Markers and goi

In [53]:
# Find all markers
cluster_markers <- cluster_markers(sc_data)

[1] "Finding all markers for time point: 23days"


Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Calculating cluster 8

Calculating cluster 9

Calculating cluster 10

Calculating cluster 11

Calculating cluster 12

Calculating cluster 13

Calculating cluster 14

Calculating cluster 15

Calculating cluster 16

Calculating cluster 17

Calculating cluster 18

Calculating cluster 19

Calculating cluster 20

Calculating cluster 21

Calculating cluster 22

Calculating cluster 23

Calculating cluster 24



In [66]:
# Find differentially expressed genes
print(genes_of_interest)
de_genes <- de.genes(genes_of_interest)

[1] "Finding differentially expressed genes for time point: 23days"
    p_val avg_log2FC pct.1 pct.2 p_val_adj cluster gene
CIT     0   3.193657 0.291 0.025         0      10  CIT


### Re-load data

In [15]:
sc_data <- load.clusters(1,1)
summary(sc_data)
head(sc_data)

Length  Class   Mode 
     1 Seurat     S4 

,orig.ident,nCount_RNA,nFeature_RNA,RNA_snn_res.1,seurat_clusters
,<fct>,<dbl>,<int>,<fct>,<fct>
1_AAACGAACACGTGAGA-1_1_23d,1_1_23d,3724,536,5,5
1_AAACGAAGTCGAAACG-1_1_23d,1_1_23d,9184,2310,5,5
1_AAACGAAGTGGCAGAT-1_1_23d,1_1_23d,6357,2068,19,19
1_AAACGAAGTGGTTCTA-1_1_23d,1_1_23d,3468,1292,17,17
1_AAACGAAGTTGGCCGT-1_1_23d,1_1_23d,1809,832,19,19
1_AAACGAATCCTTCACG-1_1_23d,1_1_23d,7587,1985,5,5
1_AAACGCTCAAGCGGAT-1_1_23d,1_1_23d,5924,1600,9,9
1_AAACGCTCACTTCAAG-1_1_23d,1_1_23d,2264,1110,3,3
1_AAACGCTCATCATCCC-1_1_23d,1_1_23d,9277,2441,19,19


In [17]:
cluster_markers <- load.markers(1)
summary(cluster_markers)
head(cluster_markers)

     p_val             avg_log2FC          pct.1            pct.2       
 Min.   :0.0000000   Min.   : 0.2500   Min.   :0.1660   Min.   :0.0000  
 1st Qu.:0.0000000   1st Qu.: 0.4057   1st Qu.:0.3320   1st Qu.:0.1670  
 Median :0.0000000   Median : 0.6149   Median :0.4510   Median :0.2750  
 Mean   :0.0001977   Mean   : 0.9433   Mean   :0.5144   Mean   :0.3416  
 3rd Qu.:0.0000000   3rd Qu.: 1.0466   3rd Qu.:0.6580   3rd Qu.:0.4570  
 Max.   :0.0099990   Max.   :13.8465   Max.   :1.0000   Max.   :0.9990  
                                                                        
   p_val_adj            cluster          gene          
 Min.   :0.0000000   10     : 1775   Length:19666      
 1st Qu.:0.0000000   20     : 1569   Class :character  
 Median :0.0000000   7      : 1219   Mode  :character  
 Mean   :0.1123661   4      : 1214                     
 3rd Qu.:0.0000035   23     : 1180                     
 Max.   :1.0000000   24     : 1140                     
                     (Ot

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
SFRP1,0,1.4141370,0.966,0.539,0,0,SFRP1
LINC01551,0,1.5938643,0.764,0.341,0,0,LINC01551
FABP7,0,1.1959399,0.931,0.530,0,0,FABP7
C1orf61,0,1.0454598,0.932,0.541,0,0,C1orf61
TTYH1,0,0.9771904,0.981,0.635,0,0,TTYH1
ID4,0,1.3536159,0.977,0.633,0,0,ID4


### continue

## other

In [67]:
for (time_point in 1:8) {
    # Load the data
    sc_data <- load.data(time_point)

    # Run PCA and clustering
    sc_data <- PCA.cluster(sc_data, res = 1)
    
    # Find all markers
    cluster_markers <- cluster.markers(sc_data)
    
    # Find differentially expressed genes
    de_genes <- de.genes(genes_of_interest)
}

[1] "Loading data for time point: 23days"


Normalizing layer: counts

Finding variable features for layer data

Centering and scaling data matrix



[1] "Running PCA and clustering for time point: 23days"


Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 29736
Number of edges: 1146654

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8914
Number of communities: 25
Elapsed time: 6 seconds

   0    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15 
5195 2314 2018 2001 1731 1659 1558 1476 1446 1341 1213 1185 1021  914  872  585 
  16   17   18   19   20   21   22   23   24 
 578  545  526  488  401  362  116  106   85 
[1] "Finding all markers for time point: 23days"


...

[1] "Running PCA and clustering for time point: 6month"


Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 123765
Number of edges: 4127244

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8981
Number of communities: 33
Elapsed time: 42 seconds

    0     1     2     3     4     5     6     7     8     9    10    11    12 
10998 10702  9669  7258  6548  6121  6007  5257  4971  4968  4336  4171  4034 
   13    14    15    16    17    18    19    20    21    22    23    24    25 
 3948  3688  3611  3253  3235  3216  2454  2172  1956  1845  1797  1552  1291 
   26    27    28    29    30    31    32 
 1158   821   800   677   609   525   117 
[1] "Finding all markers for time point: 6month"


Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Calculating cluster 8

Calculating cluster 9

Calculating cluster 10

Calculating cluster 11

Calculating cluster 12

Calculating cluster 13

Calculating cluster 14

Calculating cluster 15

Calculating cluster 16

Calculating cluster 17

Calculating cluster 18

Calculating cluster 19

Calculating cluster 20

Calculating cluster 21

Calculating cluster 22

Calculating cluster 23

Calculating cluster 24

Calculating cluster 25

Calculating cluster 26

Calculating cluster 27

Calculating cluster 28

Calculating cluster 29

Calculating cluster 30

Calculating cluster 31

Calculating cluster 32



[1] "Finding differentially expressed genes for time point: 6month"
                p_val avg_log2FC pct.1 pct.2     p_val_adj cluster   gene
SRCIN1   2.278055e-80  0.4779396 0.261 0.074  5.959165e-76      27 SRCIN1
SRCIN11 3.216065e-113  1.3244443 0.345 0.074 8.412904e-109      31 SRCIN1


In [34]:
# Plotting
plotting <- function(sc_data, genes_oi){
  # Plot the expression of the genes of interest
  for (gene in genes_oi) {
    p1 <- FeaturePlot(sc_data, features = gene, cols = c("lightgrey", "blue"), pt.size = 0.5) + 
      ggtitle(paste("Expression of", gene)) + 
      theme_minimal()
    
    # Save the plot
    ggsave(filename = paste(name_new_dir_partial, "/FeaturePlot_", gene, "_", timepoints[time_point], ".png", sep=""), plot = p1)
  }
}

In [35]:
plotting(sc_data, housekeeping_genes)
# Plot the expression of the genes of interest
plotting(sc_data, genes_of_interest)
# Plot the expression of the housekeeping genes and genes of interest together
plotting(sc_data, c(housekeeping_genes, genes_of_interest))

Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image


In [12]:
sc_data <- load.clusters(1,1)
cluster_markers <- load.markers(1)